In [1]:
useMock=False

In [2]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [3]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [4]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

In [5]:
newExChangeInfo,infopayload = AppExchangeInfo(id=b'net.maidsafe.tutorials.nodejs',scope=b'0.0.1',
                                              name=b'PyTest never ever',vendor=b'riddim from over there')

In [6]:
ffi.string(newExChangeInfo.vendor)

b'riddim from over there'

In [7]:
ffi.string(newExChangeInfo.name)

b'PyTest never ever'

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [8]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

In [9]:
newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [10]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

In [11]:
newPermissions,permission_addon=ContainerPermissions(b'_pictures',newPermissionSet)

In [12]:
newPermissions,permission_addon=ContainerPermissions(b'_pictures')

In [13]:
newPermissions.access.insert

True

In [14]:
ffi.string(newPermissions.cont_name)

b'_pictures'

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [15]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

In [16]:
myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'nobody',app_container=True)

In [17]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [18]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [19]:
ffi.string(myAuth.app.name)

b'PyTest'

In [20]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [21]:
encodedAuth=None

In [22]:
@ffi.callback("void(void *, FfiResult *, unsigned long int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [23]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [24]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
4078155827
b'AAAAADO4E_MAAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0BgAAAAAAAABub2JvZHkBAAAAAAAAAAA'


In [25]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAADO4E_MAAAAACAAAAAAAAAByaWRzVGVzdAEMAAAAAAAAAHB5dGhvbnNjcmlwdAYAAAAAAAAAUHlUZXN0BgAAAAAAAABub2JvZHkBAAAAAAAAAAA'

In [26]:
import base64

In [27]:
schemeName=b'safe-'+base64.encodebytes(ffi.string(myAuth.app.id))[:-2]

In [28]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [29]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [30]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'PyTest'
b'nobody'
b'pythonscript'


In [31]:
import os
print(os.getcwd())

/home/yomuga/maidsafe/pySafe/pySafe


In [32]:
pathToHandler=(os.getcwd()+'/uriHandler.py').encode()

In [33]:
pathToHandler

b'/home/yomuga/maidsafe/pySafe/pySafe/uriHandler.py'

In [34]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [35]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [36]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [37]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [38]:
ffi.string(myAuth.app.name)

b'PyTest'

In [39]:
ffi.string(myAuth.app.vendor)

b'nobody'

In [40]:
ffi.string(myAuth.app.name)

b'PyTest'

In [41]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

lib2.open_uri(uri,user_data,openerFeedback)

In [42]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [43]:
import time

In [44]:
import safeUtils

In [45]:
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    
    printCrustConfig(auth_granted.bootstrap_config)
    
    print('o_auth')
    print(userData)
    print(req_id)
    returnDict = ffi.from_handle(userData)
    #returnDict['AuthGranted'] = picData
    returnDict['AuthGranted'] = safeUtils.copy(auth_granted,ffi)


In [46]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [47]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [48]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [49]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [50]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [51]:
with open('answer','r') as f:
    answer=f.readlines()

In [52]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [53]:
my=data.findall(answer[-1])

In [54]:
msg = ffi.new('char[]',my[-1].encode())

In [55]:
my[-1]

'AQAAAERm9F4AAAAAAAAAACAAAAAAAAAAbhJf-9V6xLIwtiYS1eDRWmvSSRSBSUm2bmaPvH9TQoYgAAAAAAAAANEHn6z_mvlCCQG3gMz4x9JQos2yaD1oeFP-sy_2mSwbIAAAAAAAAABwVihtc2ek186DLfUojF5nHDrNoktCNBa19ABTzMMVlkAAAAAAAAAA1jQqhkRHuVtu26YtPkQlCps-xjZLOPJTf-GxyMrkBb1wVihtc2ek186DLfUojF5nHDrNoktCNBa19ABTzMMVliAAAAAAAAAA0HNx5-_v7_uZdui9JexQ6_vlVMfswycj2jZLiaErJQEgAAAAAAAAAPW3Qr7LuwRb5L768yIJMLqvPVCKwgzhGs2IjLBDS2qyGQAAAAAAAAAQAAAAAAAAADE3OC42Mi43Ni44OjU0ODMTAAAAAAAAADEzOC42OC4xODUuMjE4OjU0ODMSAAAAAAAAADEzOC42OC4xODEuNTc6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4MS42MDo1NDgzEgAAAAAAAAAxMzguNjguMTgxLjg2OjU0ODMSAAAAAAAAADEzOC42OC4xODEuODc6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNjg6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzY6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzk6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODA6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDM6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDk6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4OS4xNDo1NDgzEgAAAAAAAAAxMzguNjguMTg5LjE1OjU0ODMSAAAAAAAAADEzOC42OC4xODkuMTc6NTQ4MxIAAAA

In [57]:
returnDict={}

In [58]:
myHand=ffi.new_handle(returnDict)

In [59]:
lib.decode_ipc_msg(msg,myHand,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

[25, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 49, 55, 56, 46, 54, 50, 46, 55, 54, 46, 56, 58, 53, 52, 56, 51, 19, 0, 0, 0, 0, 0, 0, 0, 49, 51]
o_auth
<cdata 'void *' 0x7fb93820f868>
1593075268


#time.sleep(3)
lib.decode_ipc_msg(msg,ffi.new_handle(returnDict),o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

In [60]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 14 bytes>]])}

In [64]:
grantedAuth,grantedAuth_data = returnDict['AuthGranted']

TypeError: cdata 'AuthGranted *' does not support iteration

In [71]:
ffi.sizeof(ffi.new('short int[1]',[1]))

2

In [64]:
len(ffi.new('unsigned char[1]'))

1

In [72]:
ffi.new('short int[1]',[1])[0]

1

In [66]:
ffi.sizeof(ffi.new('short int[2]', [2,3]))

4

In [77]:
ffi.sizeof(ffi.new('FfiResult *'))

8

In [80]:
ffi.sizeof(ffi.new('FfiResult *')[0])

16

In [61]:
returnDict['ffi']

In [62]:
ffi

In [63]:
returnDict['ffi'].typeof('FfiResult *')

<ctype 'FfiResult *'>

In [60]:
print(returnDict['ffi'].typeof(data) == returnDict['ffi'].typeof('FfiResult *'))

TypeError: <class '_sre.SRE_Pattern'>

In [ ]:
returnDict

In [1]:
1+1

2

In [ ]:
lib.mdata_put()

In [68]:
ls

answer                        interface.py     __pycache__/
connection.py                 localization.py  pySafeUtils.py
extracted_headers/            logging.py       safe_ffi_interface.py
firstAuthenticationPoC.ipynb  log.toml         safeUtils.py
__init__.py                   picData.jpeg     uriHandler.py


In [69]:
with open('picData.jpeg','rb') as f:
    picData = f.read()

In [71]:
with open('picOut.jpeg','wb') as f:
    f.write(picData)

In [ ]:
picDataHandle=ffi.new_handle(picData)

----

now the app

In [60]:
myApp = None

In [61]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [62]:
myApp

In [63]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    global myApp
    if app != ffi.NULL:
        myApp = ffi.new('App *',app[0])
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['app'] = safeUtils.copy(app,ffi)

lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

In [64]:
grantedAuth.bootstrap_config

<cdata 'unsigned char *' 0x56351bf54540>

In [65]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 0 bytes>,
   []])}

returnDict={}

In [66]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.new_handle(returnDict),disconnect_app_registered,result_app_registered)

results
error code: 
0


In [67]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 0 bytes>,
   []]),
 'app': (<cdata 'void * *' owning 8 bytes>, [])}

In [68]:
myApp=returnDict['app']

TypeError: index() takes at least 1 argument (0 given)

In [ ]:
lib.app_account_info()

In [73]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [63]:
grantedAuth.bootstrap_config = bootstrap_config

In [64]:
grantedAuth.bootstrap_config

<cdata 'unsigned char *' 0x55b1e87cdb20>

In [65]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
error code: 
0


In [65]:
myApp[0]

<cdata 'void *' 0x55acbc6d6c20>

In [163]:
ffi.sizeof(grantedAuth.access_container_entry.containers)

8

In [166]:
ffi.sizeof(grantedAuth.bootstrap_config)

8

In [170]:
grantedAuth.bootstrap_config_len

16

In [182]:
test=ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

In [205]:
getCopy(grantedAuth)

(<cdata 'AuthGranted *' owning 336 bytes>,
 <cdata 'char[]' buffer len 16 from 'bytes' object>)

In [204]:
def getCopy(data):
    if ffi.typeof(grantedAuth) == ffi.typeof(ffi.new('AuthGranted *')):
        newData = ffi.new('AuthGranted *',data[0])
        bootstrapData = ffi.new('unsigned char[]',ffi.buffer(data.bootstrap_config,int(data.bootstrap_config_len))[:])
        newData.bootstrap_config = bootstrapData
        return newData,bootstrapData

In [282]:
test=ffi.new('short int[]',[1,4])

In [285]:
test=ffi.new('short int *',1)

In [284]:
test

<cdata 'short[]' owning 4 bytes>

In [272]:
ffi.sizeof(test)

8

In [286]:
ffi.buffer(test,ffi.sizeof(test))[:]=ffi.buffer(test,ffi.sizeof(test))[:]

b'\x01\x00\x00\x00\x00\x00\x00\x00'

In [274]:
ffi.alignof(ffi.typeof(test))

8

In [308]:
ffi.new(ffi.typeof(grantedAuth),grantedAuth[0])

<cdata 'AuthGranted *' owning 336 bytes>

In [295]:
grantedAuth.access_container_entry.containers

<cdata 'ContainerInfo *' 0x8>

In [295]:
grantedAuth.access_container_info.

<cdata 'ContainerInfo *' 0x8>

In [295]:
grantedAuth.access_container_entry.containers[0]

<cdata 'ContainerInfo *' 0x8>

In [300]:
ffi.string(ffi.new('char[]',b'haha'))

b'haha'

In [68]:
def getCopy(data):
    if ffi.typeof(data) == ffi.typeof(ffi.new('FfiResult *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        if data.description != ffi.NULL:
            description = ffi.new('char[]',ffi.string(data.description))
        else:
            description = ffi.NULL
        newData.description = description
        
        return newData,[description]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('File *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        user_metadata_ptr = ffi.new('uint8_t[]',data.user_metadata_len)
        copyLen=int(ffi.sizeof(data.user_metadata_ptr)*data.user_metadata_len)
        ffi.buffer(user_metadata_ptr,copyLen)[:]=ffi.buffer(data.user_metadata_ptr,copyLen)[:]
        newData.user_metadata_ptr = user_metadata_ptr
        
        return newData,[user_metadata_ptr]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AppExchangeInfo *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        id = ffi.new('char[]',ffi.string(data.id))
        scope = ffi.new('char[]',ffi.string(data.scope))
        name = ffi.new('char[]',ffi.string(data.name))
        vendor = ffi.new('char[]',ffi.string(data.vendor))
        newData.id = id
        newData.scope = scope
        newData.name = name
        newData.vendor = vendor
        
        return newData,[id, scope, name, vendor]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainerPermissions *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        cont_name = ffi.new('char[]',ffi.string(data.cont_name))
        newData.cont_name = cont_name
        
        return newData,[cont_name]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AuthReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerPermissions[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        containerNames=[]
        for i in range(data.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.containers[i].cont_name)))
            newData.containers[i].cont_name=containerNames[-1]
        
        return newData,[containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainersReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerPermissions[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        containerNames=[]
        for i in range(data.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.containers[i].cont_name)))
            newData.containers[i].cont_name=containerNames[-1]
        
        return newData,[containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ShareMDataReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        mdata = ffi.new('ShareMData[]',data.mdata_len)
        copyLen=int(ffi.sizeof(data.mdata)*data.mdata_len)
        ffi.buffer(mdata,copyLen)[:]=ffi.buffer(data.mdata,copyLen)[:]
        newData.mdata = mdata
        
        return newData,[mdata]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainerInfo *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        newData.name = name
        
        return newData,[name]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AccessContainerEntry *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerInfo[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        
        return newData,[containers]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AuthGranted *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        bootstrap_config = ffi.new('unsigned char[]',data.bootstrap_config_len)
        copyLen=int(ffi.sizeof(data.bootstrap_config)*data.bootstrap_config_len)
        ffi.buffer(bootstrap_config,copyLen)[:]=ffi.buffer(data.bootstrap_config,copyLen)[:]
        newData.bootstrap_config = bootstrap_config
        
        containers = ffi.new('ContainerInfo[]',data.access_container_entry.containers_len)
        copyLen=int(ffi.sizeof(data.access_container_entry.containers)*data.access_container_entry.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.access_container_entry.containers,copyLen)[:]
        newData.access_container_entry.containers = containers
        containerNames=[]
        for i in range(data.access_container_entry.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.access_container_entry.containers[i].name)))
            newData.access_container_entry.containers[i].name=containerNames[-1]
        
        return newData,[bootstrap_config,containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AppAccess *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        app_id = ffi.new('char[]',ffi.string(data.app_id))
        newData.name = name
        newData.app_id = app_id
        
        return newData,[name, app_id]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MetadataResponse *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        description = ffi.new('char[]',ffi.string(data.description))
        newData.name = name
        newData.description = description
        
        return newData,[name, description]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataKey *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        val = ffi.new('uint8_t[]',data.val_len)
        copyLen=int(ffi.sizeof(data.val)*data.val_len)
        ffi.buffer(val,copyLen)[:]=ffi.buffer(data.val,copyLen)[:]
        newData.val = val
        
        return newData,[val]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataValue *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        content = ffi.new('uint8_t[]',data.content_len)
        copyLen=int(ffi.sizeof(data.content)*data.content_len)
        ffi.buffer(content,copyLen)[:]=ffi.buffer(data.content,copyLen)[:]
        newData.content = content
        
        return newData,[content]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataEntry *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        val = ffi.new('uint8_t[]',data.key.val_len)
        copyLen=int(ffi.sizeof(data.key.val)*data.key.val_len)
        ffi.buffer(val,copyLen)[:]=ffi.buffer(data.key.val,copyLen)[:]
        newData.key.val = val
        
        content = ffi.new('uint8_t[]',data.value.content_len)
        copyLen=int(ffi.sizeof(data.value.content)*data.value.content_len)
        ffi.buffer(content,copyLen)[:]=ffi.buffer(data.value.content,copyLen)[:]
        newData.value.content = content
        
        return newData,[val,content]
    
    else:
        newData = ffi.new(ffi.typeof(data),data[0])
        return newData,[]

In [66]:
import safeUtils

In [ ]:
safeUtils.

In [68]:
import importlib

In [72]:
importlib.reload(safeUtils)

ModuleNotFoundError: No module named 'pySafe'

In [70]:
safeUtils.copy(grantedAuth)

CDefError: cannot parse "FfiResult *"
<cdef source string>:2:11: before: *

In [69]:
getCopy(grantedAuth)

(<cdata 'AuthGranted *' owning 336 bytes>,
 [<cdata 'unsigned char[]' owning 16 bytes>,
  <cdata 'ContainerInfo[]' owning 0 bytes>,
  []])

In [208]:
ffi.unpack('int[]',ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]))

TypeError: unpack() argument 1 must be _cffi_backend.CData, not str

In [208]:
ffi.unpack('int[]',ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]))

TypeError: unpack() argument 1 must be _cffi_backend.CData, not str

In [193]:
ffi.typeof(grantedAuth) == ffi.typeof(ffi.new('AuthGranted *'))

True

In [194]:
ffi.typeof(grantedAuth.app_keys) == ffi.typeof(ffi.new('AuthGranted *'))

False

'AuthGranted *'

In [184]:
ffi.cast('int[]',test)

TypeError: cannot cast to ctype 'int[]'

In [199]:
grantedAuth.bootstrap_config[0]

0

In [201]:
ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [195]:
ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

<cdata 'char[]' buffer len 16 from 'bytes' object>

In [179]:
ffi.new('unsigned char[]',ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

<cdata 'unsigned char[]' owning 17 bytes>

In [215]:
ffi.new('unsigned char[]',16)

<cdata 'unsigned char[]' owning 16 bytes>

In [211]:
ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [ ]:
ffi.sizeof()

In [67]:
grantedAuth.bootstrap_config_cap

16

In [209]:
ffi.unpack(grantedAuth.bootstrap_config,16)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [157]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [159]:
ffi.

(['AccessContInfo',
  'AccessContainerEntry',
  'AccountInfo',
  'App',
  'AppAccess',
  'AppExchangeInfo',
  'AppKeys',
  'AsymNonce',
  'AsymPublicKey',
  'AsymSecretKey',
  'AuthGranted',
  'AuthReq',
  'CipherOptHandle',
  'ContainerInfo',
  'ContainerPermissions',
  'ContainersReq',
  'EncryptPubKeyHandle',
  'EncryptSecKeyHandle',
  'FfiResult',
  'File',
  'FileContextHandle',
  'MDataAction',
  'MDataEntriesHandle',
  'MDataEntry',
  'MDataEntryActionsHandle',
  'MDataInfo',
  'MDataKey',
  'MDataPermissionsHandle',
  'MDataValue',
  'MetadataResponse',
  'ObjectHandle',
  'PermissionSet',
  'SEReaderHandle',
  'SEWriterHandle',
  'SelfEncryptorReaderHandle',
  'SelfEncryptorWriterHandle',
  'ShareMData',
  'ShareMDataReq',
  'SignPubKeyHandle',
  'SignPublicKey',
  'SignSecKeyHandle',
  'SignSecretKey',
  'SymNonce',
  'SymSecretKey',
  'UserPermissionSet',
  'XorNameArray',
  'int16_t',
  'int32_t',
  'int64_t',
  'int8_t',
  'int_fast16_t',
  'int_fast32_t',
  'int_fast64_t'

In [158]:
grantedAuth

SyntaxError: invalid syntax (<ipython-input-158-8cc55b270ca7>, line 1)

In [94]:
lib.app_registered(myAuth.app.name,testDataAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
140247862083583
b'Serialisation error'


In [148]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

In [149]:
printCrustConfig(testDataAuth.bootstrap_config)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [136]:
lali=[]
for i in range(16):
    lali.append(testDataAuth.bootstrap_config[i])
print(lali)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [71]:
testDataAuth.bootstrap_config_len

16

In [134]:
testDataAuth=ffi.new('AuthGranted *',grantedAuth[0])

In [93]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [63]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x7ffd1d44c6e0>

In [64]:
type(grantedAuth.access_container_info.tag)

int

In [65]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x7ffd1d44c6e0>

In [66]:
grantedAuth.access_container_info.tag

15000

In [67]:
grantedAuth.access_container_entry.containers_len

0

In [68]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x7ffd1d44c708>

In [69]:
grantedAuth.bootstrap_config_len

16

In [70]:
grantedAuth.bootstrap_config_cap

16

---

In [65]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [66]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x560214c901e0>

In [67]:
type(grantedAuth.access_container_info.tag)

int

In [68]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x560214c901e0>

In [69]:
grantedAuth.access_container_info.tag

15000

In [70]:
grantedAuth.access_container_entry.containers_len

0

In [71]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x560214c90208>

In [72]:
grantedAuth.bootstrap_config_len

16

In [73]:
grantedAuth.bootstrap_config_cap

16

## looking into the granted authentication

In [58]:
def printArray(array):
    arr=[]
    for element in array:
        arr.append(element)
    print(arr)

In [59]:
grantedAuth.app_keys.owner_key

<cdata 'unsigned char[32]' 0x7ffea2566a90>

In [56]:
len(grantedAuth.app_keys.owner_key)

32

In [59]:
printArray(grantedAuth.app_keys.owner_key)

[176, 107, 43, 207, 210, 127, 0, 0, 8, 235, 13, 192, 210, 127, 0, 0, 176, 107, 43, 207, 210, 127, 0, 0, 48, 238, 38, 207, 210, 127, 0, 0]


In [62]:
len(grantedAuth.app_keys.enc_key)

32

In [61]:
len(grantedAuth.app_keys.sign_pk)

32

In [62]:
len(grantedAuth.app_keys.sign_sk)

64

In [63]:
len(grantedAuth.app_keys.enc_pk)

32

In [64]:
len(grantedAuth.app_keys.enc_sk)

32